In [79]:
from pyspark.sql import SparkSession


In [80]:
#spark.stop()


In [81]:
spark = SparkSession.builder.appName("first_paquet").getOrCreate()
df = spark.read.parquet("../data/dataset.parquet")
df.printSchema()
df.show(5)


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+------

In [82]:
df.show(5)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [83]:
print(df.count())
len(df.columns)

3475226


20

In [84]:
type(df)

pyspark.sql.dataframe.DataFrame

# Count the number of null values in each columns

In [85]:
from pyspark.sql.functions import col, sum
df.select([sum(col(c).isNull().cast("int")).alias(c + "_nulles") for c in df.columns]).show()



+---------------+---------------------------+----------------------------+----------------------+--------------------+-----------------+-------------------------+-------------------+-------------------+-------------------+------------------+------------+--------------+-----------------+-------------------+----------------------------+-------------------+---------------------------+------------------+-------------------------+
|VendorID_nulles|tpep_pickup_datetime_nulles|tpep_dropoff_datetime_nulles|passenger_count_nulles|trip_distance_nulles|RatecodeID_nulles|store_and_fwd_flag_nulles|PULocationID_nulles|DOLocationID_nulles|payment_type_nulles|fare_amount_nulles|extra_nulles|mta_tax_nulles|tip_amount_nulles|tolls_amount_nulles|improvement_surcharge_nulles|total_amount_nulles|congestion_surcharge_nulles|Airport_fee_nulles|cbd_congestion_fee_nulles|
+---------------+---------------------------+----------------------------+----------------------+--------------------+-----------------+----

# check for outliers

In [86]:
import pyspark as ps
# pdf['fare_amount'].plot.box()


# Deal with null values

In [87]:
df.na.drop().show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [88]:
df.count()

3475226

# Create dure_trajet Column

In [89]:
type(df)

pyspark.sql.dataframe.DataFrame

In [90]:
import pyspark.sql.functions as sf
df = df.withColumn("dure_trajet", (sf.unix_timestamp(sf.col("tpep_dropoff_datetime"))- sf.unix_timestamp(sf.col("tpep_pickup_datetime")))/60)

In [91]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|       dure_trajet|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|    

In [92]:
df.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+------------------+--------------------+-------------------+-------------------+-------------------+
|summary|          VendorID|   passenger_count|    trip_distance|       RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|        dure_trajet|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+---------

# Delete columns  with the same value duplicates over the columns 

In [100]:
df.select("trip_distance").distinct().count()

4545

In [115]:
for col in df.columns:
    val= df.select(col).distinct().count()
    rows_count= df.count()
    if val <= 4:
      print(col)
      print(val)    #    print(col)

VendorID
4


store_and_fwd_flag
3
improvement_surcharge
4
congestion_surcharge
4
cbd_congestion_fee
3


# Select the Categorial Columns

In [119]:
categrial_df= df.select("VendorID", "RatecodeID", "store_and_fwd_flag","PULocationID","DOLocationID","payment_type")
categrial_df.show(2)

+--------+----------+------------------+------------+------------+------------+
|VendorID|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|
+--------+----------+------------------+------------+------------+------------+
|       1|         1|                 N|         229|         237|           1|
|       1|         1|                 N|         236|         237|           1|
+--------+----------+------------------+------------+------------+------------+
only showing top 2 rows



# Analyse the relation between Categorial data and the target

# Select the Numerical Features

In [122]:
numerical_df= df.select("dure_trajet","passenger_count", "trip_distance", "fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","Airport_fee","cbd_congestion_fee")
numerical_df.show(2)

+-----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|dure_trajet|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+-----------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       8.35|              1|          1.6|       10.0|  3.5|    0.5|       3.0|         0.0|                  1.0|        18.0|                 2.5|        0.0|               0.0|
|       2.55|              1|          0.5|        5.1|  3.5|    0.5|      2.02|         0.0|                  1.0|       12.12|                 2.5|        0.0|               0.0|
+-----------+---------------+-------------+-----------+-----+-------+----------+------------+--